## Generating Benchmark Data for Wikipedia Dataset

The data is taken from the [Web Traffic Time Series Forecasting
](https://www.kaggle.com/competitions/web-traffic-time-series-forecasting/overview) challenge on Kaggle, a research competiton organized by Google ($25,000).

In [1]:
ROOT = "/Users/liaopeiyuan/Documents/"
data_dir = ROOT + "web-traffic-time-series-forecasting/"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from tqdm import tqdm

/opt/anaconda3/envs/test/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv(data_dir + 'train_1.csv').fillna(0)
df

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145058,Underworld_(serie_de_películas)_es.wikipedia.o...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13.0,12.0,13.0,3.0,5.0,10.0
145059,Resident_Evil:_Capítulo_Final_es.wikipedia.org...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145060,Enamorándome_de_Ramón_es.wikipedia.org_all-acc...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145061,Hasta_el_último_hombre_es.wikipedia.org_all-ac...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
import dask.dataframe as dd

ddf = dd.from_pandas(df, npartitions=10000)

forecast_horizon = 100

def predict(row):
    data = pd.DataFrame({'ds': (row.index)[1:][-forecast_horizon:], 'y':(row.values)[1:][-forecast_horizon:]})
    m = Prophet()
    m.fit(data)
    
    future = m.make_future_dataframe(periods=forecast_horizon)
    future.tail()
    forecast = m.predict(future)

    return row.Page, forecast[['yhat']][-forecast_horizon:].values.reshape(-1)

In [14]:
dask_series = ddf.apply(predict, axis=1, meta=('float', 'object'))  

In [15]:
result = dask_series.compute()

00:07:11 - cmdstanpy - INFO - Chain [1] start processing
00:07:11 - cmdstanpy - INFO - Chain [1] start processing
00:07:11 - cmdstanpy - INFO - Chain [1] start processing
00:07:11 - cmdstanpy - INFO - Chain [1] start processing
00:07:11 - cmdstanpy - INFO - Chain [1] start processing
00:07:11 - cmdstanpy - INFO - Chain [1] start processing
00:07:11 - cmdstanpy - INFO - Chain [1] start processing
00:07:11 - cmdstanpy - INFO - Chain [1] start processing
00:07:11 - cmdstanpy - INFO - Chain [1] done processing
00:07:11 - cmdstanpy - INFO - Chain [1] done processing
00:07:11 - cmdstanpy - INFO - Chain [1] done processing
00:07:11 - cmdstanpy - INFO - Chain [1] done processing
00:07:11 - cmdstanpy - INFO - Chain [1] done processing
00:07:11 - cmdstanpy - INFO - Chain [1] done processing
00:07:11 - cmdstanpy - INFO - Chain [1] done processing
00:07:11 - cmdstanpy - INFO - Chain [1] done processing
00:07:23 - cmdstanpy - INFO - Chain [1] start processing
00:07:23 - cmdstanpy - INFO - Chain [1]

00:08:55 - cmdstanpy - INFO - Chain [1] done processing
00:08:55 - cmdstanpy - INFO - Chain [1] start processing
00:08:56 - cmdstanpy - INFO - Chain [1] done processing
00:08:56 - cmdstanpy - INFO - Chain [1] start processing
00:08:56 - cmdstanpy - INFO - Chain [1] done processing
00:08:56 - cmdstanpy - INFO - Chain [1] start processing
00:08:57 - cmdstanpy - INFO - Chain [1] start processing
00:08:57 - cmdstanpy - INFO - Chain [1] done processing
00:08:57 - cmdstanpy - INFO - Chain [1] done processing
00:08:57 - cmdstanpy - INFO - Chain [1] start processing
00:08:57 - cmdstanpy - INFO - Chain [1] done processing
00:08:57 - cmdstanpy - INFO - Chain [1] start processing
00:08:57 - cmdstanpy - INFO - Chain [1] done processing
00:08:57 - cmdstanpy - INFO - Chain [1] start processing
00:08:57 - cmdstanpy - INFO - Chain [1] done processing
00:09:07 - cmdstanpy - INFO - Chain [1] start processing
00:09:07 - cmdstanpy - INFO - Chain [1] done processing
00:09:07 - cmdstanpy - INFO - Chain [1] 

In [20]:
import pickle
pickle.dump(result[0], open('result.pkl', 'wb'))

In [ ]:
forecast_horizon = 100
node_list = []
forecasts = np.zeros((len(df), forecast_horizon)).astype(float)

In [ ]:
for i, ro in tqdm(df.iterrows(), total=len(df)):
    node_list.append(ro.Page)
    data = pd.DataFrame({'ds': (ro.index)[1:], 'y':(ro.values)[1:]})
    m = Prophet()
    m.fit(data)
    
    future = m.make_future_dataframe(periods=forecast_horizon)
    future.tail()
    forecast = m.predict(future)

    forecasts[i,:] = forecast[['yhat']][-forecast_horizon:].values.reshape(-1)